# JIRA Data Gathering

In [1]:
from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth("tpilvelis@ymail.com", "<token-id>")

In [2]:
import requests
import json

#Globals
nodes = []
node_types = [] #Epic, Story, Task, Test
parent_child_pairs = []

def get_nodes(JSONString):
    r = json.loads(JSONString)

    curr = r["key"]
    children = []

    childrenJSON = r["fields"]["issuelinks"]
    for child in childrenJSON:
        if "outwardIssue" in child:
            childURL = child["outwardIssue"]["self"]
            childKey = child["outwardIssue"]["key"]
            
            children.append(childKey)
            parent_child_pairs.append( (curr, childKey) )
            
            make_request(childURL)

    nodes.append(curr)
    node_types.append(r["fields"]["issuetype"]["name"])

def make_request(url):
    
    headers = {
    "Accept": "application/json"
    }

    response = requests.request("GET", url, headers=headers, auth=auth )

    JSONResponse = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
    get_nodes(JSONResponse) 

#Root
make_request("https://tstframework1.atlassian.net/rest/api/3/issue/TSTFRAM-1")

## Data Catagory Transformation

In [3]:
color_list = []

for item in node_types:
    col = None
    if 'Test' == item:
        col = "#e84118" #Default Red as failure
    elif 'Task' == item:
        col = "#00a8ff"
    elif 'Story' == item:
        col = "#273c75"
    elif 'Epic' == item:
        col = "#9c88ff"
    else:
        raise NotImplementedError
    color_list.append(col)

## Visualizations

In [4]:
from pyvis.network import Network

net = Network(notebook=True)

net.add_nodes(nodes, color=color_list)
net.add_edges(parent_child_pairs)

net.show("this.html")